<a href="https://colab.research.google.com/github/GenaroHacker/actionable_builder/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



Database Structure


```
# Table: PLACES
# (ID INTEGER PRIMARY KEY, NAME TEXT UNIQUE)

# Table: CONTEXTS
# (ID INTEGER PRIMARY KEY, PLACE_ID INTEGER, NAME TEXT, RANK INTEGER, FOREIGN KEY(PLACE_ID) REFERENCES PLACES(ID), UNIQUE(PLACE_ID, NAME))

# Table: TAGS
# (ID INTEGER PRIMARY KEY, CONTEXT_ID INTEGER, NAME TEXT, RANK INTEGER, FOREIGN KEY(CONTEXT_ID) REFERENCES CONTEXTS(ID), UNIQUE(CONTEXT_ID, NAME))

# Table: TASKS
# (ID INTEGER PRIMARY KEY, TAG_ID INTEGER, NAME TEXT, RANK INTEGER, FOREIGN KEY(TAG_ID) REFERENCES TAGS(ID), UNIQUE(TAG_ID, NAME))

# Table: STEPS
# (ID INTEGER PRIMARY KEY, TASK_ID INTEGER, NAME TEXT, ORDER_SEQUENCE INTEGER, MINUTES INTEGER, FOREIGN KEY(TASK_ID) REFERENCES TASKS(ID), UNIQUE(TASK_ID, ORDER_SEQUENCE))

# Table: PLACEHOLDERS
# (ID INTEGER PRIMARY KEY, TYPE TEXT, VALUE TEXT, RANK INTEGER, UNIQUE(TYPE, VALUE))

# Table: HISTORY
# (ID INTEGER PRIMARY KEY, STEP_ID INTEGER, ACTION TEXT, TIME TEXT, FOREIGN KEY(STEP_ID) REFERENCES STEPS(ID))

```




In [1]:
%%capture
#@title Set Up
!git clone https://github.com/GenaroHacker/write_imports.git
from write_imports.write_imports import write_imports

!git clone https://github.com/GenaroHacker/actionable_builder.git


In [2]:

# Delete this line to see the new imports
%%capture



#@title Import Statements
#Modules: ['write_imports', 'actionable_builder']



from write_imports.write_imports import write_imports

from write_imports.LearningPython.BasicExamples.intro_functions import print_hello_world

from actionable_builder.modules.rank_columns import ask_user_cmp
from actionable_builder.modules.rank_columns import ask_user_cmp
from actionable_builder.modules.rank_columns import check_minimum_tasks
from actionable_builder.modules.rank_columns import extract_placeholders
from actionable_builder.modules.rank_columns import check_placeholders
from actionable_builder.modules.rank_columns import fetch_records_for_ranking
from actionable_builder.modules.rank_columns import update_rank
from actionable_builder.modules.rank_columns import sort_and_update

from actionable_builder.modules.add_example_data import add_example_data

from actionable_builder.modules.create_database import create_table
from actionable_builder.modules.create_database import insert_record
from actionable_builder.modules.create_database import create_empty_database
from actionable_builder.modules.create_database import check_or_insert_with_parent
from actionable_builder.modules.create_database import check_step_exists
from actionable_builder.modules.create_database import add_task
from actionable_builder.modules.create_database import add_placeholders
from actionable_builder.modules.create_database import initialize_database
from actionable_builder.modules.create_database import create_example_database
from actionable_builder.modules.create_database import download_database

from actionable_builder.modules.sql_core import insert_several_records
from actionable_builder.modules.sql_core import read_records
from actionable_builder.modules.sql_core import read_last_record
from actionable_builder.modules.sql_core import update_record
from actionable_builder.modules.sql_core import remove_record
from actionable_builder.modules.sql_core import run_command

from actionable_builder.modules.pick_zipfian import pick_zipfian





print(write_imports([]))





In [3]:
# @title Create and Populate Database

if __name__ == "__main__":
    initialize_database()

    # Example task input
    task = {
        "place": "home",
        "context": "create",
        "tag": "guitar",
        "task": "loop %note%",
        "steps": [
            "identify the note in each string",
            "start the metronome at 30",
            "loop over each string 3 times"
        ]
    }

    add_task(task)

    # Example usage of add_placeholders
    placeholders = {
        "note": ["C", "C#", "D"],
        "color": ["red", "blue", "green"]
    }

    add_placeholders(placeholders)



Choose an option:
1. Create empty database
2. Create database with example data
3. Upload database
2


In [4]:
# @title Rank Columns

# This script updates ranks in 'tasks.db' tables.
# It groups unranked records by parent ID, sorts them based on user input and updates their ranks.

if __name__ == "__main__":
    db_path = '/content/tasks.db'

    check_minimum_tasks(db_path)
    check_placeholders(db_path)

    # Sorting and updating ranks for each table, grouped by their parent column
    for table_name, parent_column in [('CONTEXTS', 'PLACE_ID'), ('TAGS', 'CONTEXT_ID'),
                                      ('TASKS', 'TAG_ID'), ('PLACEHOLDERS', 'TYPE')]:
        sort_and_update(db_path, table_name, parent_column)

    download_database()



Sorting for table: CONTEXTS
Sorting group: 1
Sorting group: 2
Sorting group: 3
Sorting for table: TAGS
Sorting group: 1
Sorting group: 2
Sorting group: 3
Sorting group: 4
Sorting group: 5
Sorting group: 6
Sorting for table: TASKS
Sorting group: 1
Sorting group: 2
Sorting group: 3
Sorting group: 4
Sorting group: 5
Sorting group: 6
Sorting group: 7
Sorting group: 8
Sorting group: 9
Sorting group: 10
Sorting group: 11
Sorting group: 12
Sorting group: 13
Sorting group: 14
Sorting group: 15
Sorting group: 16
Sorting group: 17
Sorting group: 18
Sorting for table: PLACEHOLDERS
Sorting group: book
Sorting group: color
Sorting group: duration
Sorting group: equipment
Sorting group: event
Sorting group: flavor
Sorting group: herb
Sorting group: instrument
Sorting group: item
Sorting group: material
Sorting group: note
Sorting group: period
Sorting group: plant
Sorting group: time
Sorting group: topic


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
import sqlite3
import random

# Refactoring the function to be integrated into a final program
def select_zipf_tasks(cursor):
    """
    Selects 5 unique tasks for the selected place in the database using Zipf distribution.
    Replaces placeholders in task names and returns a list of dicts with task id and text.

    Args:
    cursor (sqlite3.Cursor): A cursor to execute queries on the database.

    Returns:
    list of dicts: A list of 5 dictionaries with keys 'task id' and 'text'.
    """

    def select_by_zipf(items):
        """
        Applies Zipf's law to select an item based on rank.
        """
        if not items:
            return None

        harmonic_sum = sum(1.0 / i for i in range(1, len(items) + 1))
        probabilities = [(1.0 / i) / harmonic_sum for i in range(1, len(items) + 1)]
        return random.choices(items, weights=probabilities, k=1)[0]

    def replace_placeholders(task_name, placeholders):
        """
        Replaces placeholders in a task name using Zipf distribution.
        """
        while '%' in task_name:
            start = task_name.find('%')
            end = task_name.find('%', start + 1)
            placeholder_type = task_name[start + 1:end]
            relevant_placeholders = [p for p in placeholders if p[1] == placeholder_type]
            placeholder_value = select_by_zipf(relevant_placeholders)[2] if relevant_placeholders else ''
            task_name = task_name[:start] + placeholder_value + task_name[end + 1:]
        return task_name

    # Select the place marked as selected
    cursor.execute("SELECT * FROM PLACES WHERE IS_SELECTED = 1;")
    selected_place = cursor.fetchone()
    if not selected_place:
        return "No place is selected."

    selected_place_id = selected_place[0]
    selected_tasks = set()
    final_tasks = []

    # Fetch all placeholders for replacement
    cursor.execute("SELECT * FROM PLACEHOLDERS;")
    placeholders = cursor.fetchall()

    while len(final_tasks) < 5:
        # Context, Tag, and Task selection
        cursor.execute(f"SELECT * FROM CONTEXTS WHERE PLACE_ID = {selected_place_id};")
        contexts = cursor.fetchall()
        selected_context = select_by_zipf(contexts)
        cursor.execute(f"SELECT * FROM TAGS WHERE CONTEXT_ID = {selected_context[0]};")
        tags = cursor.fetchall()
        selected_tag = select_by_zipf(tags)
        cursor.execute(f"SELECT * FROM TASKS WHERE TAG_ID = {selected_tag[0]};")
        tasks = cursor.fetchall()
        selected_task = select_by_zipf(tasks)

        # Ensure uniqueness of tasks
        if selected_task[0] not in selected_tasks:
            selected_tasks.add(selected_task[0])
            task_with_placeholders_replaced = replace_placeholders(selected_task[2], placeholders)
            final_tasks.append({"task id": selected_task[0], "text": task_with_placeholders_replaced})

    return final_tasks

# Example of how to use the function
# Create a database connection and cursor
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Execute the function
tasks_output = select_zipf_tasks(cursor)

# Closing the database connection
conn.close()

tasks_output



[{'task id': 16, 'text': 'plant lavender in the garden'},
 {'task id': 11, 'text': 'practice piano for 15 minutes'},
 {'task id': 10, 'text': 'create a photo frame using cardboard'},
 {'task id': 21, 'text': 'loop D'},
 {'task id': 2, 'text': 'paint a landscape using blue colors'}]